In [35]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Don't show debug info while training
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import datetime
from tensorflow.keras import layers, models
import pandas as pd
import shutil
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.cluster import AgglomerativeClustering
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, Conv1D, LSTM, Dense, Flatten, Bidirectional, BatchNormalization, Activation, LayerNormalization, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Add, Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, GRU, LayerNormalization, ReLU, Input, Conv1D, LSTM, TimeDistributed, Dense, Activation, MultiHeadAttention, Attention, Bidirectional, RepeatVector, Masking, Multiply, Layer, Conv1DTranspose, Concatenate, ZeroPadding1D, Cropping1D, Lambda
from tensorflow.keras import regularizers
from tensorflow.keras import Input, Model
from tensorflow.keras.losses import CategoricalCrossentropy
import sys
import warnings
import json
import optuna
import random
import matplotlib.gridspec as gridspec
import os
from tensorflow.keras.models import load_model
import shap
from itertools import product
from tqdm import tqdm
import umap
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding
from sklearn.decomposition import PCA, FactorAnalysis,  DictionaryLearning, NMF
from Bio import motifs
from tensorflow.keras.preprocessing.sequence import pad_sequences
from Bio import motifs
from Bio.Seq import Seq
import h5py
from modisco import tfmodisco_workflow
import subprocess
from IPython.display import HTML

**Validate the input data and custom variables to make sure the file exists, that the input columns exist, that the sequences and groups are in the right format, that all other custom variables are the right type and that sequences are not too long and there are not too many different groups.**

In [36]:
def validate_config_and_data(config):
    
    # Ensure 'full_file' is defined in custom_config
    if 'full_file' not in config:
        raise ValueError("'full_file' must be defined in the custom configuration.")
    
    # Load the dataset to perform checks
    try:
        data = pd.read_csv(config['full_file'], sep=config['sep'])
    except Exception as e:
        raise ValueError(f"Failed to load the data file {config['full_file']}: {e}")
    
    # Check if the required columns are present
    missing_columns = []
    for col in [config['sequence_column_name'], config['group_column_name']]:
        if col not in data.columns:
            missing_columns.append(col)
    if missing_columns:
        raise ValueError(f"The following required columns are missing from the data file: {', '.join(missing_columns)}. Please check the column names or the separator and try again.")

    
    # Check the sequence column for valid nucleotide strings
    if not data[config['sequence_column_name']].apply(lambda x: isinstance(x, str) and all(c in 'ATCGN' for c in x)).all():
        raise ValueError(f"The sequence column '{config['sequence_column_name']}' contains invalid nucleotide strings.")
    
    # Check the group column for less than 10 unique values
    if data[config['group_column_name']].nunique() >= 10:
        raise ValueError(f"The group column '{config['group_column_name']}' must have less than 10 unique values.")
    
    # Check for the length of the longest sequence
    max_sequence_length = data[config['sequence_column_name']].str.len().max()
    print(f"The longest sequence is {max_sequence_length} nucleotides long.")
    if max_sequence_length >= 100000:
        warnings.warn("Warning: The longest sequence is 100,000 or more nucleotides long. This is not a problem but can cause extremly high memory usage - check your available vram.", RuntimeWarning)
    elif 10000 <= max_sequence_length < 100000:
        warnings.warn("Mild Warning: The longest sequence is between 10,000 and 100,000 nucleotides long. This is not a problem but can cause extensive memory usage - check your available vram.", RuntimeWarning)

**Load the default configuration and the custom configuration file. Merge both to main configuration file. Create the main out directory if it is not yet present.**

In [37]:
def create_main_dir(config):
    os.makedirs(config['out_dir'], exist_ok=True)

**Create the main directory of a single 'Run' based on current time. Copy the notebook inside to serve as a backup save. Split the main file to training and validation files in a 'datasets' subdirectory.**

In [38]:
def create_out_dir_and_copy_source_file_in(config):
    # Create a directory inside "Models" based on the current timestamp
    current_time = datetime.datetime.now().strftime("%d%m%Y_%H%M%S%f")[:18]
    if config["OPTIMISE"]:
        directory_name = f"{config['out_dir']}/Optimising_Run_{current_time}"
    else:
        directory_name = f"{config['out_dir']}/Individual_Run_{current_time}"
    os.makedirs(directory_name, exist_ok=True)
    
    print(f"Out dir: {directory_name}")

    # Copy the source file to the new directory to save as a reference
    source_file_path = "main_guide.ipynb"
    destination_file_path = os.path.join(directory_name, "main_guide.ipynb")
    shutil.copy(source_file_path, destination_file_path)
    
    return directory_name

def prepare_train_test(config, directory_name):
    
    full_dataframe = pd.read_csv(config['full_file'], sep=config['sep'])
    
    train, temp = train_test_split(full_dataframe, test_size=config['test_and_validation_size']*2, stratify=full_dataframe[config['group_column_name']], random_state=42)
    test, validation = train_test_split(temp, test_size=0.5, stratify=temp[config['group_column_name']], random_state=42)

    os.makedirs(f"{directory_name}/Created_Datasets", exist_ok=True)
    
    full_csv_out = f"{directory_name}/Created_Datasets/original_file.bed"
    full_dataframe.to_csv(full_csv_out, sep=config['sep'], index=False)
    
    training_csv_out = f"{directory_name}/Created_Datasets/training.bed"
    train.to_csv(training_csv_out, sep=config['sep'], index=False)

    validation_csv_out = f"{directory_name}/Created_Datasets/validation.bed"
    validation.to_csv(validation_csv_out, sep=config['sep'], index=False)
    
    test_csv_out = f"{directory_name}/Created_Datasets/test.bed"
    test.to_csv(test_csv_out, sep=config['sep'], index=False)
    
def prepare_the_necessary_directories_and_raw_files(config):
    validate_config_and_data(config)
    create_main_dir(config)
    directory_name = create_out_dir_and_copy_source_file_in(config)
    prepare_train_test(config, directory_name)
    return directory_name

**Set up the data encoding and processing functions in order to enable procedural batch generation from raw files represented as a RepeatDataset structure. The sequences and groups are proceduraly one-hot encoded and padded to the longest sequence.**

In [87]:
def parse_categories(group, categories_list):
    # Convert the category to a one-hot encoded vector
    category_index = tf.argmax(tf.equal(categories_list, group))
    one_hot_encoded_group = tf.one_hot(category_index, depth=len(categories_list))
    return one_hot_encoded_group

def one_hot_encode(sequence, longest_sequence, encoding=['A', 'C', 'G', 'T']):
    # Define the mapping for nucleotides to integers
    nucleotide_to_index = tf.constant(encoding)
    # Split the sequence into characters
    nucleotides = tf.strings.bytes_split(sequence)
    # Find the indices of each nucleotide in the sequence
    indices = tf.argmax(tf.equal(tf.expand_dims(nucleotides, -1), nucleotide_to_index), axis=-1)
    # Perform one-hot encoding
    one_hot_encoded = tf.one_hot(indices, depth=4)
    # Pad the sequence to the desired width
    padded_sequence = tf.pad(one_hot_encoded, paddings=[[0, longest_sequence - tf.shape(one_hot_encoded)[0]], [0, 0]], constant_values=0)
    # Ensure the sequence is trimmed to `longest_sequence` in case it's longer
    padded_sequence = padded_sequence[:longest_sequence, :]
    return padded_sequence

def parse_crosslink_scores_sparse(scores_str, longest_sequence):
    """Parse the sparse encoded crosslink scores and create a list of scores with the given sequence length."""

    def handle_empty_or_invalid():
        # Return a tensor of zeros with the given sequence length
        return tf.zeros([longest_sequence], dtype=tf.float32)

    def handle_valid_scores():
        # Process the valid scores_str
        score_pairs = tf.strings.split(scores_str, ';')
        indices_scores = tf.strings.split(score_pairs, ':')
        indices_scores = indices_scores.to_tensor()
        indices = tf.strings.to_number(indices_scores[:, 0], out_type=tf.int32)
        scores = tf.strings.to_number(indices_scores[:, 1], out_type=tf.float32)
        
        full_scores = tf.zeros([longest_sequence], dtype=tf.float32)
        full_scores = tf.tensor_scatter_nd_update(full_scores, tf.expand_dims(indices, 1), scores)
        return full_scores

    # Check if the scores_str contains a ":"
    contains_colon = tf.strings.regex_full_match(scores_str, ".*:.*")

    return tf.cond(contains_colon, handle_valid_scores, handle_empty_or_invalid)

def combined_encode(sequence, scores_str_list, longest_sequence):
    # One-hot encode the sequence
    one_hot_encoded_sequence = one_hot_encode(sequence, longest_sequence)
    # Parse the crosslink scores for each clip column
    crosslink_scores_list = [parse_crosslink_scores_sparse(scores_str, longest_sequence) for scores_str in scores_str_list]
    # Expand the scores to have a last dimension size of 1 to match the sequence
    crosslink_scores_expanded = [tf.expand_dims(scores, axis=-1) for scores in crosslink_scores_list]
    # Concatenate the one-hot encoded sequence and all the scores
    combined_encoded = tf.concat([one_hot_encoded_sequence] + crosslink_scores_expanded, axis=-1)
    return combined_encoded

def process_line(line, config, column_indices, categories_list, longest_sequence):
    fields = tf.io.decode_csv(line, record_defaults=[[""]] * len(column_indices), field_delim=config['sep'])
    sequence = fields[column_indices[config['sequence_column_name']]]
    groups = fields[column_indices[config['group_column_name']]]
    scores_str_list = []
    if config['signal_column_name']:
        scores_str_list = [fields[column_indices[clip_column]] for clip_column in config['signal_column_name']]
    encoded_sequence_with_scores = combined_encode(sequence, scores_str_list, longest_sequence)
    encoded_groups = parse_categories(groups, categories_list)
    
    return encoded_sequence_with_scores, encoded_groups

def shuffle_file(file_path, out_dir_for_scrambled_data):
    # Specify the path to your file
    shuffled_file_path = f"{out_dir_for_scrambled_data}/original_file_shuffled.bed"
    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Separate the header and the rows
    header = lines[0]
    rows = lines[1:]
    # Shuffle the rows
    random.shuffle(rows)
    # Write the header and the shuffled rows back to a new file
    with open(shuffled_file_path, 'w') as file:
        file.write(header)  # Write the header first
        file.writelines(rows)  # Then write the shuffled rows
    file_path = shuffled_file_path
    return file_path

def encode_from_csv(file_path, config, column_indices, longest_sequence, categories_list, out_dir_for_scrambled_data=None):
    if out_dir_for_scrambled_data:
        file_path = shuffle_file(file_path, out_dir_for_scrambled_data)
            
    dataset = tf.data.TextLineDataset(file_path).skip(1)  # Skip header
    
    dataset = dataset.map(lambda line: process_line(line, config, column_indices, categories_list, longest_sequence),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(config['batch_size'])
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset


**Prepare the information needed to encode the raw data in a RepeatDataset structure. Store the raw validation and training files in a ReturnDataset structure that contains the logic to encode the sequences and groups as needed. Get the shape of first RepeatDataset data batch. Calculate the number of steps needed to process the entire training and validation datasets.**

In [40]:
def get_column_indices_max_length_and_categories(config):
    # Read the first line to get column names
    with tf.io.gfile.GFile(config['full_file'], 'r') as f:
        column_names = f.readline().strip().split(config['sep'])
        
    column_indices = {name: index for index, name in enumerate(column_names)}
    # Initialize to find the longest sequence and unique groups
    longest_sequence = 0
    unique_groups = set()

    with tf.io.gfile.GFile(config['full_file'], 'r') as f:
        next(f)  # Skip header line
        for line in f:
            fields = line.strip().split(config['sep'])
            sequence = fields[column_indices[config['sequence_column_name']]]
            group = fields[column_indices[config['group_column_name']]]
            longest_sequence = max(longest_sequence, len(sequence))
            unique_groups.add(group)

    # Sort the unique groups to ensure consistency
    categories_list = tf.constant(sorted(unique_groups))

    return column_indices, longest_sequence, categories_list

def get_shapes_of_inputs(validation_dataset):
    # Get the first batch from the dataset
    first_item_encoded = next(iter(validation_dataset.take(1)))
    
    # Extract features and labels from the first batch
    features, labels = first_item_encoded
    
    # Convert TensorFlow tensors to NumPy arrays and get shapes
    features_shape = features.numpy().shape
    labels_shape = labels.numpy().shape

    return features_shape, labels_shape

def prepare_the_data_for_training(config, directory_name):
    
    column_indices, longest_sequence, categories_list = get_column_indices_max_length_and_categories(config)
    
    training_csv_out = f"{directory_name}/Created_Datasets/training.bed"
    training_dataset = encode_from_csv(training_csv_out, config, column_indices, longest_sequence, categories_list).repeat()
    
    validation_csv_out = f"{directory_name}/Created_Datasets/validation.bed"
    validation_dataset = encode_from_csv(validation_csv_out, config, column_indices, longest_sequence, categories_list).repeat()
    
    # Get the lengths of the CSV files without the header
    training_length = sum(1 for _ in open(training_csv_out)) - 1
    validation_length = sum(1 for _ in open(validation_csv_out)) - 1
    
    # Calculate the steps per epoch for training and validation
    training_steps = training_length // config['batch_size'] + 1
    validation_steps = validation_length // config['batch_size'] + 1
    
    # Get the shapes of the inputs
    features_shape, labels_shape = get_shapes_of_inputs(validation_dataset)
    return training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape

**Build the model based on the hyperparematers in a config file. The architecture can be partially customised form the custom_config file. For advanced use this can also be adapted, but it should serve as a good starting point for sequence classification tasks as is.**

In [41]:
def build_model(config, input_shape, output_shape):
    original_input = Input(shape=(input_shape[1], input_shape[2]))
    x = original_input
    
    for block in range(config['num_blocks']):
        x = Conv1D(filters=int(config['Final_CNN_units'] * ((1-config['CNN_Units_Increase_By_Percent']) ** (config['num_blocks']-1-block))), 
                   kernel_size=config['kernel_size']*((config['Increase_Kernel_By']) ** block), 
                   padding="same", 
                   kernel_regularizer=regularizers.l2(l2=config['l2_lambda']), 
                   dilation_rate=((config['Increase_Dilation_By']) ** block)
                   )(x)
        
        if config['normalization'] == 'BatchNormalization':
            lstm_output = BatchNormalization()(lstm_output)
        elif config['normalization'] == 'LayerNormalization':
            lstm_output = LayerNormalization()(lstm_output)
               
        x = Activation('relu')(x)
        x = Dropout(rate=config['dropout_prob'])(x)  
        x = MaxPooling1D(pool_size=config['reduce_by'])(x)
        
    conv_output = x
    
    lstm_output = Bidirectional(GRU(units=config['LSTM_units'], 
                        return_sequences=False, 
                        kernel_regularizer=regularizers.l2(l2=config['l2_lambda']))
                    )(conv_output)
    
    if config['normalization'] == 'BatchNormalization':
        lstm_output = BatchNormalization()(lstm_output)
    elif config['normalization'] == 'LayerNormalization':
        lstm_output = LayerNormalization()(lstm_output)
    
    if config['pooling_type'] == 'average':
        conv_output_pooled = GlobalAveragePooling1D()(conv_output)
    elif config['pooling_type'] == 'max':
        conv_output_pooled = GlobalMaxPooling1D()(conv_output)
        
    x = Concatenate()([lstm_output, conv_output_pooled])      
    
    
    x = Dropout(rate=config['dropout_prob'])(conv_output_pooled)  
    
    for i in range(2):
        x = Dense(units=config['Dense_units']//(2**i), 
                activation='relu', 
                kernel_regularizer=regularizers.l2(l2=config['l2_lambda'])
                )(x)
        
        x = Dropout(rate=config['dropout_prob'])(x)  
    
    output = Dense(units=output_shape[1], activation='softmax')(x)
    
    model = Model(inputs=original_input, outputs=output)
    
    return model

**Validate the model size to make sure the memory usage does not excede the gpu vram limit. This is very much an estimation and should not be treated as a fact.**

In [42]:
def estimate_model_memory_usage_from_layers(config, input_shape, output_shape, dtype=np.float32):
    
    dtype_size = np.dtype(dtype).itemsize
    total_weight_memory = 0
    total_activation_memory = 0
    
    # Assuming the input shape includes the batch size
    current_shape = input_shape
    
    # Conv1D + BatchNorm + Activation + MaxPooling for each block
    for block in range(config['num_blocks']):
        filters = int(config['Final_CNN_units'] * ((1-config['CNN_Units_Increase_By_Percent']) ** (config['num_blocks']-1-block)))
        # Weight memory for Conv1D: kernel_size * current_shape[-1] * filters + filters (for bias)
        total_weight_memory += ((config['kernel_size']*((config['Increase_Kernel_By']) ** block)) * current_shape[-1] * filters + filters) * dtype_size
        
        # Output shape after Conv1D
        current_shape = (current_shape[0], current_shape[1], filters)
        # Activation memory is for the output volume
        total_activation_memory += np.prod(current_shape) * dtype_size
        
        # MaxPooling
        current_shape = (current_shape[0], current_shape[1] // config['reduce_by'], current_shape[2])
    
    # LSTM Layer
    lstm_param_count = 4 * (config['LSTM_units'] * current_shape[-1] + config['LSTM_units'] * config['LSTM_units'] + config['LSTM_units'])  # For both directions
    total_weight_memory += lstm_param_count * dtype_size
    if config['pooling_type'] is not None:
        
        # Assuming return_sequences is True, so the output shape is the same
        total_activation_memory += np.prod((current_shape[0], current_shape[1], config['LSTM_units'] * 2)) * dtype_size
        # Flatten
        current_shape = (current_shape[0], np.prod(current_shape[1:]))
    else:
        total_activation_memory += np.prod((current_shape[0], config['LSTM_units'] * 2)) * dtype_size
        current_shape = (current_shape[0], config['LSTM_units'] * 2)
    
    # Dense Layer
    # For Dense: current_shape[-1] * Dense_units + Dense_units (for bias)
    total_weight_memory += (current_shape[-1] * config['Dense_units'] + config['Dense_units']) * dtype_size
    total_activation_memory += np.prod((current_shape[0], config['Dense_units'])) * dtype_size
    
    # Output Layer
    total_weight_memory += (config['Dense_units'] * output_shape[1] + output_shape[1]) * dtype_size
    total_activation_memory += np.prod((current_shape[0], output_shape[1])) * dtype_size
    
    total_memory_bytes = total_weight_memory + total_activation_memory
    total_memory_mb = total_memory_bytes / (1024 ** 2)  # Convert to MB
    
    return total_memory_mb, total_memory_mb / 1024  # MB and GB

def validate_memory_usage_before_building_the_model(config, features_shape, labels_shape):
    # Estimate memory usage and dynamically decide the unit for display
    memory_usage_mb, memory_usage_gb = estimate_model_memory_usage_from_layers(config, features_shape, labels_shape, dtype=np.float32)
    if memory_usage_mb > 1024:  # More than 1 GB
        print(f"Estimated memory usage for a batch: {memory_usage_gb:.2f} GB")
    else:
        print(f"Estimated memory usage for a batch: {int(memory_usage_mb)} MB")
        
    if memory_usage_gb > 10:
        raise MemoryError(f"Estimated memory usage exceeds 10 GB ({memory_usage_gb:.2f} GB). Halting execution to prevent system overload.")

        
def validate_parameter_number_and_give_better_memory_estimation(model, features_shape):
    
    total_params = model.count_params()
    if total_params > 5e7:
        warnings.warn("Serious Warning: The model has over 50 million parameters. This is too much for most cases.", RuntimeWarning)
    elif total_params > 5e6:
        warnings.warn("Warning: The model has over 5 million parameters. Consider simplifying the model if training is inefficient.", RuntimeWarning)

**Train the model based on the config file and save the training history. This can also be adapted to specific use cases.**

In [43]:
# Train the model 
def train_the_model(config, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape):
    
    features_shape, labels_shape = get_shapes_of_inputs(validation_dataset)

    validate_memory_usage_before_building_the_model(config, features_shape, labels_shape)
    
    model = build_model(config, input_shape=features_shape, output_shape=labels_shape)
    
    optimizer = Adam(learning_rate=config['learning_rate'])
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
    validate_parameter_number_and_give_better_memory_estimation(model, features_shape)

    model.summary()
    
    # Early stopping for both training and validation losses
    early_stopping = EarlyStopping(monitor='val_loss', patience=config['patience'], restore_best_weights=True)    
    
    verbose = 0 if config['OPTIMISE'] else 1
        
    # Use this dictionary in the fit method
    history = model.fit(training_dataset,
                        epochs=config['epochs'],
                        verbose=verbose,
                        steps_per_epoch=training_steps,
                        validation_data=validation_dataset,
                        validation_steps=validation_steps,
                        callbacks=[early_stopping])

    return model, history

**Prepare the validation data as a numpy array - needed for the evaluation functions.**

In [74]:
def dataset_to_numpy(dataset, steps):
    
    val_dataset = dataset.take(steps)
    # To get numpy arrays from the dataset
    features_list = []
    labels_list = []

    for features, labels in val_dataset:
        features_list.append(features.numpy())
        labels_list.append(labels.numpy())

    X_val = np.concatenate(features_list, axis=0) if features_list else np.array([])
    y_val = np.concatenate(labels_list, axis=0) if labels_list else np.array([])
    
    return X_val, y_val

**Create out directories for trained model with configuration and the evaluation.**

In [45]:
def create_out_dirs(directory_name, optimise):
    # First create a general output directory
    os.makedirs(f"{directory_name}/Results", exist_ok=True)
    general_out = f"{directory_name}/Results"
    if optimise:
        model_info_out = f"{general_out}/Saved_Trained_Model_With_Best_Parameters"
        os.makedirs(model_info_out, exist_ok=True)
        model_eval_out = f"{general_out}/Evaluated_Trained_Model_With_Best_Parameters"
        os.makedirs(model_eval_out, exist_ok=True)
        optimisation_out = f"{general_out}/Optimisation_Process_and_Results"
        os.makedirs(optimisation_out, exist_ok=True)
        return model_info_out, model_eval_out, optimisation_out
    
    model_info_out = f"{general_out}/Saved_Trained_Model"
    os.makedirs(model_info_out, exist_ok=True)
    model_eval_out = f"{general_out}/Evaluated_Trained_Model"
    os.makedirs(model_eval_out, exist_ok=True)
    return model_info_out, model_eval_out

**Function to predict the validation dataset. Function to evaluate and save the model's accuracy, precision, recall, F1-score, and AUROC. Function to save a confusion matrix plot and the training curves.**

In [46]:
def predict_validation_dataset(model, X_val, y_val, batch_size):
    y_pred = np.vstack([model.predict(X_val[i:i + batch_size]) for i in range(0, len(X_val), batch_size)])
    # Get the predicted classes
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_val_classes = np.argmax(y_val, axis=1)
    return y_pred, y_pred_classes, y_val_classes

def auroc_score(y_val, y_pred, average='standard'):
    # Calculate ROC AUC for each class
    auc_scores = []
    for i in range(y_val.shape[1]):  # iterate over each class
        # Compute ROC AUC for the i-th class
        auc = roc_auc_score(y_val[:, i], y_pred[:, i])
        auc_scores.append(auc)
    if average == 'weighted':
        # Optionally, calculate a weighted average AUROC if classes are imbalanced
        weights = y_val.mean(axis=0)
        weighted_average_auc = np.average(auc_scores, weights=weights)
        return weighted_average_auc
    else:
        return np.average(auc_scores)

def save_evaluation_metrics(y_val_classes, y_pred_classes, y_val, y_pred, directory_name):
    metrics = {
        "Accuracy": accuracy_score(y_val_classes, y_pred_classes),
        "Precision": precision_score(y_val_classes, y_pred_classes, average='weighted', zero_division=0),
        "Recall": recall_score(y_val_classes, y_pred_classes, average='weighted'),
        "F1-score": f1_score(y_val_classes, y_pred_classes, average='weighted'),
        "AUROC": auroc_score(y_val, y_pred, average='weighted')
    }
    with open(os.path.join(directory_name, "evaluation_metrics.txt"), "w") as f:
        for key, value in metrics.items():
            f.write(f"{key}: {value}\n")

def save_confusion_matrix(y_val_classes, y_pred_classes, directory_name):
    # Compute the confusion matrix
    cm = confusion_matrix(y_val_classes, y_pred_classes)
    
    # Normalize the confusion matrix to percentages
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Plot and save the confusion matrix
    plt.figure(figsize=(7, 7))
    sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="YlGnBu", cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Percentages)')
    plt.savefig(os.path.join(directory_name, "confusion_matrix.png"))  # Save the plot as PNG
    plt.close()  # Close the plot

def save_training_curves(history, directory_name):
    # Get the training and validation loss and accuracy values from history
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_accuracy = history.history['auc']
    val_accuracy = history.history['val_auc']

    # Create a figure with two subplots for loss and accuracy
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Plot the loss curves on the first subplot
    ax1.plot(train_loss, label='Training Loss')
    ax1.plot(val_loss, label='Validation Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot the accuracy curves on the second subplot
    ax2.plot(train_accuracy, label='Training AUC')
    ax2.plot(val_accuracy, label='Validation AUC')
    ax2.set_title('Training and Validation AUC')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    # Save the figure containing both subplots
    plt.savefig(os.path.join(directory_name, "training_validation_curves.png"))
    plt.close(fig)  # Close the figure to free up memory

**Functions to save the model, configuration files, model summary, weights and architecture, and hyperparameters.**

In [47]:
def save_training_history(history, directory_name):
    with open(os.path.join(directory_name, "training_history.json"), "w") as f:
        # Convert possible NumPy types in the history to native Python types
        history_dict = {k: [float(val) for val in v] for k, v in history.history.items()}
        json.dump(history_dict, f, indent=4)


def save_model_summary(model, directory_name):
    with open(os.path.join(directory_name, "model_summary.txt"), "w") as f:
        # Redirect the default standard output to the file
        original_stdout = sys.stdout  # Save the original standard output
        sys.stdout = f  # Redirect to the file
        model.summary()  # This will write to the file instead of the console
        sys.stdout = original_stdout  # Reset standard output to its original value


def save_weights_and_architecture(model, directory_name):
    model.save_weights(os.path.join(directory_name, "model_weights.h5"))
    with open(os.path.join(directory_name, "model_architecture.json"), "w") as f:
        f.write(model.to_json())


def save_models_hyperparameters(model, batch_size, directory_name):
    with open(os.path.join(directory_name, "hyperparameters.txt"), "w") as f:
        f.write(str(model.get_config()))
        f.write(f"Batch Size: {batch_size}\n")
        f.write(str(model.get_config()))


def save_full_model(model, directory_name):
    model.save(os.path.join(directory_name, "full_model.h5"))
    optimizer_config = model.optimizer.get_config()
    with open(os.path.join(directory_name, "config.txt"), "w") as f:
        for key, value in optimizer_config.items():
            f.write(f"{key}: {value}\n")


def save_config(config, directory_name):
    if config['OPTIMISE']:
        with open(os.path.join(directory_name, "optimal_config.json"), "w") as f:
            json.dump(config, f, indent=4)
    else:
        with open(os.path.join(directory_name, "config.json"), "w") as f:
            json.dump(config, f, indent=4)        
        
def save_optimization_details(details, directory_name, config):
    with open(os.path.join(directory_name, 'Optimization_Results.txt'), 'w') as f:
        f.write("Optimization Results\n")
        f.write("====================\n")
        f.write(f"Best Parameters: {details['best_params']}\n")
        f.write(f"Best Accuracy: {details['best_value']}\n")
    with open(os.path.join(directory_name, 'Optimised_Config.txt'), 'w') as f:    
        f.write("Optimised Config:\n")
        f.write("====================\n")
        for key, value in config.items():
            f.write(f"{key}: {value}\n")
    with open(os.path.join(directory_name, 'Optimization_History.txt'), 'w') as f:
        f.write("Optimization History:\n")
        f.write("====================\n")
        for entry in details['history']:
            f.write(f"Trial {entry['trial_number']}: Value: {entry['value']}, Parameters: {entry['params']}\n")  
            
                  
def save_optimisation(directory_name, study, config):
    # Now, save the optimization history and best parameters
    optimization_details = {
        'best_params': study.best_params,
        'best_value': study.best_value,
        'history': [{'trial_number': trial.number, 'value': trial.value, 'params': trial.params} for trial in study.trials]
    }
    
    save_optimization_details(optimization_details, directory_name, config)

**Evaluate the trained model and save it with its configuration.**

In [48]:
def evaluate_and_save_the_model(config, model, history, directory_name, validation_dataset, validation_steps, study=None):
    # create output directory
    model_info_out, model_eval_out, *model_optimise_out  = create_out_dirs(directory_name, config["OPTIMISE"])
    # Prepare the validation dataset for evaluation
    X_val, y_val = dataset_to_numpy(validation_dataset, validation_steps)
    
    # Create the output directories
    model_info_out, model_eval_out, *model_optimise_out  = create_out_dirs(directory_name, config["OPTIMISE"])
    
     # Evaluate the model
    y_pred, y_pred_classes, y_val_classes = predict_validation_dataset(model, X_val, y_val, config['batch_size'])
    save_confusion_matrix(y_val_classes, y_pred_classes, model_eval_out)
    save_evaluation_metrics(y_val_classes, y_pred_classes, y_val, y_pred, model_eval_out)
    save_training_curves(history, model_eval_out)   
    
    # Save the model information
    save_config(config, directory_name)
    save_training_history(history, model_info_out)
    save_model_summary(model, model_info_out)    
    save_weights_and_architecture(model, model_info_out)
    save_models_hyperparameters(model, config['batch_size'], model_info_out)
    save_full_model(model, model_info_out)
    if config["OPTIMISE"]:
        save_optimisation(model_optimise_out[0], study, config)

**Define the objective function for optuna optimization. Create the search space for each parameter.**

In [49]:
def objective(trial, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape, config):
    try:
        # Define the range of hyperparameters for Optuna to explore
        config_optuna = {
            'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32, 64]),
            'num_blocks': trial.suggest_int('num_blocks', 2, 8),
            'dropout_prob': trial.suggest_float('dropout_prob', 0.0, 0.4),
            'l2_lambda': trial.suggest_loguniform('l2_lambda', 0, 1e-2),
            'reduce_by': trial.suggest_categorical('reduce_by', [2, 3, 4]),
            'Final_CNN_units': trial.suggest_categorical('Final_CNN_units', [16, 32, 64, 128, 256, 512, 1024]),
            'CNN_Units_Increase_By_Percent': trial.suggest_float('CNN_Units_Increase_By_Percent', 0.1, 0.6),
            'LSTM_units': trial.suggest_categorical('LSTM_units', [16, 32, 64, 128, 256, 512, 1024]),
            'Dense_units': trial.suggest_categorical('Dense_units', [16, 32, 64, 128, 256, 512, 1024]),
            'kernel_size': trial.suggest_categorical('kernel_size', [3, 5, 7, 9]),
            'Increase_Kernel_By': trial.suggest_int('Increase_Kernel_By', 1, 4),
            'Increase_Dilation_By': trial.suggest_int('Increase_Dilation_By', 1, 3),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
            'normalization': trial.suggest_categorical('normalization', [None, 'BatchNormalization', 'LayerNormalization']),
            'pooling_type': trial.suggest_categorical('pooling_type', [None, 'max', 'average']),
        }
        
        config_trial = {**config, **config_optuna}
        
        # Train the model with the trial's hyperparameters
        model, history = train_the_model(config_trial, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape)

        # Evaluate the model on the validation set to get the 'true' best performance
        # Note: This evaluation step is crucial if early stopping might have restored the model to a state from a previous epoch
        val_loss, val_accuracy, val_auc = model.evaluate(validation_dataset, steps=validation_steps)

        # Since we're optimizing for accuracy, return the negative of accuracy (because Optuna minimizes the objective)
        return val_accuracy
    
    except Exception as e:
        # Log the error or handle it as per your needs
        print(f"Error during trial: {e}")
        # print config_trial to see the error
        print(config_trial)
        # Return a very low accuracy value
        return -1.0
    

**Execute optuma optimisation of the hyperparameters**

In [50]:
def optimise_with_optuna(config, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape):
    study = optuna.create_study(direction='maximize')
    
    study.optimize(lambda trial: objective(trial, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape, config), n_trials=config["n_trials"])

    # Use the best hyperparameters
    best_params = study.best_trial.params
    best_value = study.best_trial.value
    print("Best hyperparameters:", best_params)
    print("Best accuracy:", best_value)
    return study, best_params   

**Explain the model by calculating shap scores for the entire dataset, for all features and groups. Creating a file original_file_shuffled_with_contribution_scores.pkl - encoded dataframe**

In [79]:
def create_model_explain_out_dir(directory_name):
    model_explanation_out = f"{directory_name}/Results/Model_Explanation"
    os.makedirs(model_explanation_out, exist_ok=True)

    return model_explanation_out

def prepare_for_shap(config, directory_name, size_of_background = 200):
    
    # Number of samples
    num_batches_needed_for_background = (size_of_background//config['batch_size']) + 1

    column_indices, longest_sequence, categories_list = get_column_indices_max_length_and_categories(config)
    full_dataset = encode_from_csv(config['full_file'], config, column_indices, longest_sequence, categories_list, out_dir_for_scrambled_data=f'{directory_name}/Created_Datasets')

    full_dataset_length = sum(1 for _ in open(config['full_file'])) - 1

    # Compute SHAP values for the desired number of batches
    num_batches_needed = (full_dataset_length//config['batch_size']) + 1

    # Select a small, representative background dataset
    print('Preparing the background data...')
    background_data, _ = dataset_to_numpy(full_dataset, num_batches_needed_for_background)
    
    return full_dataset, background_data, num_batches_needed

def compute_shaps(config, directory_name, size_of_background):
    full_dataset, background_data, num_batches_needed = prepare_for_shap(config, directory_name, size_of_background)
    if config['OPTIMISE']:
        model = load_model(f'{directory_name}/Results/Saved_Trained_Model_With_Best_Parameters/full_model.h5')
    else:
        model = load_model(f'{directory_name}/Results/Saved_Trained_Model/full_model.h5')
    # Initialize the SHAP explainer (assuming a Keras model)
    explainer = shap.GradientExplainer(model, background_data[:size_of_background], batch_size=config['batch_size'])
    all_shap_values = []

    for i, batch in tqdm(enumerate(full_dataset.take(num_batches_needed)), total=num_batches_needed, desc='Explaining Batches of Samples'):
        
        # To get numpy arrays from the dataset
        features, labels = batch
        
        features = features.numpy()
        labels = labels.numpy()
        
        # Calculate SHAP values
        shap_values_batch = explainer.shap_values(features)
        
        # turn list of numpy arrays into a numpy array
        shap_values_batch_array = np.stack(shap_values_batch)
        all_shap_values.append(shap_values_batch_array)
        
    # Concatenate the SHAP values for all batches
    all_shap_values = np.concatenate(all_shap_values, axis=1)
    return all_shap_values

def import_config(directory_name):

    config_file_path = os.path.join(directory_name, 'config.json')
    alternative_config_file_path = os.path.join(directory_name, 'optimal_config.json')

    # Check which file exists and select it
    if os.path.exists(config_file_path):
        file_to_open = config_file_path
    elif os.path.exists(alternative_config_file_path):
        file_to_open = alternative_config_file_path
    else:
        raise FileNotFoundError("Neither config.json nor optimal_config.json was found.")
    
    # Now open and load the configuration from the selected file
    with open(file_to_open) as f:
        config = json.load(f)
        
    return config

def add_shap_values_to_full_shuffled(all_shap_values, directory_name, config, model_explanation_out):
    full_shuffled = pd.read_csv(f'{directory_name}/Created_Datasets/original_file_shuffled.bed', sep=config['sep'])
    unique_groups = sorted(full_shuffled[config['group_column_name']].unique())
    for i in range(all_shap_values.shape[0]):
        column_data = all_shap_values[i]

        full_shuffled[f'Contribution_Scores_for_Group_{unique_groups[i]}'] = list(column_data)
        
    def adjust_array_lengths(row):
        sequence_length = len(row[config['sequence_column_name']])
        for i in range(len(unique_groups)):  # Assuming 3 groups, adjust as necessary
            key = f'Contribution_Scores_for_Group_{unique_groups[i]}'
            # Truncate the array to match sequence_length
            # Note: This assumes you want to truncate along the first dimension
            row[key] = row[key][:sequence_length]
        return row
    
    # Apply the function across the DataFrame
    full_shuffled = full_shuffled.apply(adjust_array_lengths, axis=1)
    full_shuffled.to_pickle(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl')

**Bin importance scores for a specific group over the sequnces in that group to see the importante features in classification of those seuqences.**

In [230]:
def create_model_explain_visualisation_dir(model_explanation_out):
    model_explanation_visualisation_out = f"{model_explanation_out}/Visualisations"
    os.makedirs(model_explanation_visualisation_out, exist_ok=True)

    return model_explanation_visualisation_out

# Create a function that scores for kmer at each position in the sequence and then bins the sequences, sums the socres for each bin and plots them.
def create_aggregate_heatmap_df_per_kmers(sequences, contribution_scores, explaining_params, model_explanation_out, config):
    print("Creating aggregate heatmap per kmers...")
    # Initial empty nucleotides list
    nucleotides = []
    
    # Check if sequence group names are present and append nucleotides
    if config.get("sequence_column_name", None) is not None:
        nucleotides.extend(['A', 'C', 'G', 'T'])
    
    # Check if kmer length is 1 and signal columns are present
    if explaining_params["kmer_length"] == 1 and config.get("signal_column_name", None) is not None:
        # Append the score column names to the nucleotides list
        score_column_names = config.get("signal_column_name", [])
        nucleotides.extend(score_column_names)
    
    # Generate all possible kmers
    kmers = [''.join(kmer) for kmer in product(nucleotides, repeat=explaining_params["kmer_length"])]
    
    # Create a dictionary to map nucleotides to indices
    nucleotide_to_index = {nucleotide: idx for idx, nucleotide in enumerate(nucleotides)}

    heatmap_df = pd.DataFrame()
    
    # For each kmer
    for n, kmer in tqdm(enumerate(kmers), total=len(kmers), desc="Processing kmers"):
        
        # Create empty lists for relative positions and scores
        relative_positions_all = []
        scores_all = []
        sequence_index = []

        # For each sequence and score
        for num, (seq, score) in tqdm(enumerate(zip(sequences, contribution_scores)), total=len(sequences), desc=f"Processing sequences for {kmer}"):

            # For each possible position of the kmer in the sequence
            for j in range(len(seq) - explaining_params["kmer_length"] + 1):

                # Calculate the mean score for this position
                if explaining_params["kmer_length"] == 1:
                    mean_score = score[j][nucleotide_to_index[kmer]]
                else:
                    mean_score = np.mean([score[j+i][nucleotide_to_index[kmer[i]]] for i in range(explaining_params["kmer_length"])])

                # Append the mean score and relative position to the lists
                scores_all.append(mean_score)
                relative_positions_all.append(j/len(seq))
                sequence_index.append(num)

        # Skip if no scores for this kmer
        if not scores_all:
            continue

        # Create a DataFrame for easy binning
        df = pd.DataFrame({
            'RelativePosition': relative_positions_all,
            f'Score_{kmer}': scores_all,
            'Seq_Index': sequence_index
        })

        # Bin the data according to relative position
        df['Bin'] = pd.cut(df['RelativePosition'], bins=explaining_params["num_bins"], labels=False)

        # Group by bin and calculate mean of scores
        df_mean_inside_seq = df.groupby(['Seq_Index', 'Bin'])[f'Score_{kmer}'].mean().reset_index()

        # Merge with the overall DataFrame
        if heatmap_df.empty:
            heatmap_df = df_mean_inside_seq
        else:
            heatmap_df = pd.merge(heatmap_df, df_mean_inside_seq, on=['Seq_Index', 'Bin'])
    
    # save the heatmap_df
    heatmap_df.to_pickle(f'{model_explanation_out}/heatmap_df.pkl')

**Turn a dataframe to an array for ease of plotting.**

In [229]:
def kmar_scores_df_to_array(heatmap_df, explaining_params):
    # Get the unique kmers
    kmers = [col.split('_')[1] for col in heatmap_df.columns if col.startswith('Score_')]
    # Find number of unique values in the column Bin
    num_bins = explaining_params["num_bins"]
    
    # Initialize the heatmap data
    heatmap_data = np.zeros((len(kmers), num_bins))

    # For each kmer
    for n, kmer in enumerate(kmers):
        if f'Score_{kmer}' in heatmap_df.columns:  
            # Group by bin and calculate mean of scores for the kmer
            df_mean_scores = heatmap_df.groupby('Bin')[f'Score_{kmer}'].mean().reset_index()
            assert len(df_mean_scores) == num_bins , "The clusters are so small that in some clusters we dont have all bins present. Reduce bin number or change dimensionality reduction method."

            # Assuming bins are 0-indexed and match directly with heatmap_data columns
            heatmap_data[n, df_mean_scores['Bin'].values] = df_mean_scores[f'Score_{kmer}'].values
            
    return heatmap_data

**Plot a heatmap and clustermap of binned importance scores over a group.**

In [228]:
def plot_a_heatmap_of_importance_scores(heatmap_df, model_explanation_visualisation_out, explaining_params, config):
    print("Plotting...")
    nucleotides = []
    
    # Check if sequence group names are present and append nucleotides
    if config.get("sequence_column_name", None) is not None:
        nucleotides.extend(['A', 'C', 'G', 'T'])
    
    # Check if kmer length is 1 and signal columns are present
    if explaining_params["kmer_length"] == 1 and config.get("signal_column_name", None) is not None:
        # Append the score column names to the nucleotides list
        score_column_names = config.get("signal_column_name", [])
        nucleotides.extend(score_column_names)
    
    # Generate all possible kmers
    kmers = [''.join(kmer) for kmer in product(nucleotides, repeat=explaining_params["kmer_length"])]
    
    heatmap_data = kmar_scores_df_to_array(heatmap_df, explaining_params)

    # Find the maximum value in the heatmap data
    absolute_maximum = np.nanmax(np.abs(heatmap_data))
    
    # Plot
    if explaining_params["kmer_length"] == 1:
        fig, ax = plt.subplots(figsize=(10, len(nucleotides)))
    else:
        fig, ax = plt.subplots(figsize=(10, 10))
        
    sns.heatmap(heatmap_data, cmap='seismic', cbar_kws={'label': 'Average Contribution Score'}, yticklabels=kmers, vmin=-absolute_maximum, vmax=absolute_maximum, ax=ax)
    ax.set_xlabel('Relative position')
    ax.set_ylabel('Kmer')

    # Set x-ticks
    x_ticks = np.linspace(0, explaining_params["num_bins"], 11) 
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([f'{i*10}%' for i in range(11)])

    # Set plot name and show the plot
    plt.title(f'Metaprofile of Average Importance Scores for Each {explaining_params["kmer_length"]}-mer for {explaining_params["group_to_explain"]} predicted genes')

    # Save plot as svg
    plt.savefig(f'{model_explanation_visualisation_out}/{explaining_params["group_to_explain"]}_group_importance_score_binned_to_{explaining_params["num_bins"]}_bins_metaplot_for_{explaining_params["kmer_length"]}mer.svg', bbox_inches='tight')
    
    # Show the plot
    plt.show()

    # Plot with clustermap
    clustermap = sns.clustermap(heatmap_data, col_cluster=False, cmap='seismic', cbar_kws={'label': 'Average Contribution Score'}, yticklabels=kmers, vmin=-absolute_maximum, vmax=absolute_maximum)

    # Relabel the axes
    clustermap.ax_heatmap.set_xlabel('Relative position')
    clustermap.ax_heatmap.set_ylabel('Kmer')

    # Set x-ticks
    x_ticks = np.linspace(0, explaining_params["num_bins"], 11)
    clustermap.ax_heatmap.set_xticks(x_ticks)
    clustermap.ax_heatmap.set_xticklabels([f'{i*10}%' for i in range(11)])

    # Set plot name
    clustermap.fig.suptitle(f'Metaprofile of Average Importance Scores for Each {explaining_params["kmer_length"]}-mer for {explaining_params["group_to_explain"]} predicted genes')

    # Save plot as svg
    plt.savefig(f'{model_explanation_visualisation_out}/{explaining_params["group_to_explain"]}_group_importance_score_binned_to_{explaining_params["num_bins"]}_bins_clustermap_for_{explaining_params["kmer_length"]}mer.svg', bbox_inches='tight')

    # Show the plot
    plt.show()

In [56]:
def dimensionality_reduction(pivot_df, explaining_params):
    if explaining_params["type_of_reduction"] == 'TSNE':
        tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
        data_reduced = tsne.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'UMAP':
        reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
        data_reduced = reducer.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'PCA':
        pca = PCA(n_components=2)
        data_reduced = pca.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'Isomap':
        pca = PCA(n_components=2)
        data_reduced = pca.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'Isomap':
        isomap = Isomap(n_components=2)
        data_reduced = isomap.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'MDS':
        mds = MDS(n_components=2)
        data_reduced = mds.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'Spectral Embedding':
        spectral_embedding = SpectralEmbedding(n_components=2)
        data_reduced = spectral_embedding.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'Factor Analysis':
        factor_analysis = FactorAnalysis(n_components=2)
        data_reduced = factor_analysis.fit_transform(pivot_df)
    if explaining_params["type_of_reduction"] == 'Dictionary Learning':
        dict_learning = DictionaryLearning(n_components=2)
        data_reduced = dict_learning.fit_transform(pivot_df)
        
    return data_reduced

In [227]:
def plot_clustered_heatmap_of_importance_scores(heatmap_df, model_explanation_visualisation_out, explaining_params, config): 
    print("Plotting...")
    nucleotides = []

    # Check if sequence group names are present and append nucleotides
    if config.get("sequence_column_name", None) is not None:
        nucleotides.extend(['A', 'C', 'G', 'T'])
    
    # Check if kmer length is 1 and signal columns are present
    if explaining_params["kmer_length"] == 1 and config.get("signal_column_name", None) is not None:
        # Append the score column names to the nucleotides list
        score_column_names = config.get("signal_column_name", [])
        nucleotides.extend(score_column_names)
    
    # Generate all possible kmers
    kmers = [''.join(kmer) for kmer in product(nucleotides, repeat=explaining_params["kmer_length"])]
    
    features = pd.DataFrame(heatmap_df.set_index(['Seq_Index', 'Bin']).stack()).reset_index()
    
    features.columns = ['Seq_Index', 'Bin', 'Nucleotide', 'Score']
    
    # Pivot this table to have a wider format suitable for UMAP and clustering
    pivot_df = features.pivot_table(index='Seq_Index', columns=['Bin', 'Nucleotide'], values='Score').fillna(0)
    
    # Perform dimensionality reduction
    data_reduced = dimensionality_reduction(pivot_df, explaining_params)
    
    # Perform clustering
    clusters_num = explaining_params["number_of_clusters"]
    aggclust = AgglomerativeClustering(n_clusters=clusters_num, distance_threshold=None, affinity='euclidean', linkage='ward')
    clusters = aggclust.fit_predict(data_reduced)

    # Add clusters back to the original DataFrame
    pivot_df['Cluster'] = clusters

    # Set up the plot
    plt.figure(figsize=(12, 10))

    # Scatter plot of the UMAP embeddings, colored by cluster assignment
    # Note: Clusters labeled '-1' are considered outliers by HDBSCAN
    sns.scatterplot(x=data_reduced[:, 0], y=data_reduced[:, 1], hue=clusters, palette="Spectral", legend="full", s=50)

    plt.title(f'{explaining_params["type_of_reduction"]} projection of the dataset, colored by AgglomerativeClustering clusters')
    plt.xlabel(f'{explaining_params["type_of_reduction"]} Dimension 1')
    plt.ylabel(f'{explaining_params["type_of_reduction"]} Dimension 2')
    plt.legend(title='Cluster')

    # Remove legend title for aesthetics if desired
    plt.legend(title='Cluster').get_title().set_fontsize('14')

    # Save the plot as SVG
    plt.savefig(f'{model_explanation_visualisation_out}/{explaining_params["type_of_reduction"]}_projection_of_{explaining_params["group_to_explain"]}_group_importance_score_binned_to_{explaining_params["num_bins"]}_bins_for_{explaining_params["kmer_length"]}_clustered_into_{clusters_num}_clusters.svg', bbox_inches='tight')
    plt.show()
    
    
    # Chunk of code to add the clustering infomation to the heatmap_df
    pivot_df_info = pivot_df.reset_index()[['Seq_Index', 'Cluster']]
    cluster_series = pivot_df_info.iloc[:, -1]
    clusters_info = pd.DataFrame({
        'Seq_Index': cluster_series.index,
        'Cluster': cluster_series.values
    })
    heatmap_df_with_clusters = pd.merge(heatmap_df, clusters_info, on='Seq_Index')
    
    # Adjust the figure size as necessary
    fig, axs = plt.subplots(clusters_num, 1, figsize=(13, clusters_num * (4+len(score_column_names))))
    fig.suptitle(f'Metaprofile of Average Importance Scores for Each {explaining_params["kmer_length"]}-mer for {explaining_params["group_to_explain"]} predicted genes', fontsize=10)
    
    # If there's only one cluster, wrap axs in a list to make it iterable.
    if clusters_num == 1:
        axs = [axs]
        
    absolute_maximum = 0    
    
    for cluster in range(clusters_num):
        cluster_data = heatmap_df_with_clusters[heatmap_df_with_clusters['Cluster'] == cluster]
        
        heatmap_data = kmar_scores_df_to_array(cluster_data, explaining_params)

        # Find the maximum value in the heatmap data
        absolute_maximum = max(np.nanmax(np.abs(heatmap_data)), absolute_maximum)    
        
    for cluster in range(clusters_num):
        cluster_data = heatmap_df_with_clusters[heatmap_df_with_clusters['Cluster'] == cluster]
        
        heatmap_data = kmar_scores_df_to_array(cluster_data, explaining_params)
            
        # Plotting adjustments for subplots
        ax = axs[cluster]  # Select the current Axes object for the cluster
        sns.heatmap(heatmap_data, cmap='seismic', cbar_kws={'label': 'Average Contribution Score'}, yticklabels=kmers, vmin=-absolute_maximum, vmax=absolute_maximum, ax=ax)
        ax.set_title(f'Cluster {cluster}')  # Set title with cluster number
        ax.set_xlabel('Relative position')
        ax.set_ylabel('Kmer')
        
        # Set x-ticks
        x_ticks = np.linspace(0, explaining_params["num_bins"], 11) 
        ax.set_xticks(x_ticks)
        ax.set_xticklabels([f'{i*10}%' for i in range(11)])

    plt.subplots_adjust(hspace=0.4, wspace=0.2)  # Add space between plots
    # Adjust layout and spacing
    fig.suptitle(f'Metaprofile of Average Importance Scores for Each {explaining_params["kmer_length"]}-mer for {explaining_params["group_to_explain"]} predicted genes', y=1.02)

    # Save the plot as SVG
    plt.savefig(f'{model_explanation_visualisation_out}/{explaining_params["type_of_reduction"]}_projection_of_{explaining_params["group_to_explain"]}_group_importance_score_binned_to_{explaining_params["num_bins"]}_bins_for_{explaining_params["kmer_length"]}mer_clustered_into_{clusters_num}_clusters.svg', bbox_inches='tight')
    plt.show()

In [243]:
def draw_importance_scores_values(shap_values, index, model_explanation_visualisation_out, explaining_params, config):
    sequence_column_name = config.get("sequence_column_name")
    signal_column_names = config.get("signal_column_name", [])
    
    colors = ['blue', 'orange', 'green', 'red']
    nucleotide_mapping = ['A', 'C', 'G', 'U']
    
    # Get the absolute max value from the SHAP values array
    absolute_max_value = np.abs(shap_values).max()

    # Set up the figure and gridspec
    fig = plt.figure(figsize=(16, 4+len(signal_column_names)))
    gs = gridspec.GridSpec(2, 2, width_ratios=[50, 1], height_ratios=[4, len(signal_column_names)])

    # Check if sequence column is present and plot
    if sequence_column_name is not None:
        ax_sequence = plt.subplot(gs[0, 0])
        for idx in range(4):
            ax_sequence.bar(range(shap_values.shape[0]), shap_values[:, idx], color=colors[idx], label=f'{nucleotide_mapping[idx]}', alpha=1)
        
        ax_sequence.set_xlim(0, shap_values.shape[0])
        ax_sequence.set_ylim(-absolute_max_value, absolute_max_value) 
        ax_sequence.set_xlabel('Sequence position', fontsize=16)
        ax_sequence.set_ylabel('Importance Score', fontsize=16)
        ax_sequence.set_title(f'Importance Scores for {explaining_params["group_to_plot"]} group over sequence with index {explaining_params["sequence_indexes"][index]}', fontsize=16)
        ax_sequence.legend(nucleotide_mapping, loc='upper left')

    # Check if signal columns are present and plot as heatmap
    if signal_column_names:
        ax_heatmap = plt.subplot(gs[1, 0])
        ax_cbar = plt.subplot(gs[1, 1])
        num_signals = len(signal_column_names)
        signal_data = shap_values[:, 4:4+num_signals]

        sns.heatmap(signal_data.T, cmap='seismic', cbar=True, ax=ax_heatmap, cbar_ax=ax_cbar, yticklabels=signal_column_names, annot=False, vmin=-absolute_max_value, vmax=absolute_max_value)

        ax_heatmap.set_xlabel('Sequence position', fontsize=16)
        ax_heatmap.set_ylabel('Signal Columns', fontsize=16)
        ax_heatmap.set_title(f'Signal Values for {explaining_params["group_to_plot"]} group over sequence with index {explaining_params["sequence_indexes"][index]}', fontsize=16)
        ax_heatmap.set_yticklabels(signal_column_names, rotation=0, fontsize=13)  # Keep y-ticks horizontal for better readability

    plt.subplots_adjust(hspace=0.5, wspace=0.2)  # Add space between plots
    plt.tight_layout()
    plt.savefig(f'{model_explanation_visualisation_out}/importance_scores_for_{explaining_params["group_to_plot"]}_group_over_sequence_with_index_{explaining_params["sequence_indexes"][index]}.svg', bbox_inches='tight')
    plt.show()


In [241]:
def check_column_config(explaining_params, config):
    # Error handling for invalid configurations
    if explaining_params["kmer_length"] > 1:
        if config.get("sequence_column_name", None) is None:
            raise ValueError("There should be a sequence column present if the kmer_length is above 1.")
        if config.get("signal_column_name", None) is not None:
            raise ValueError("There should not be any score columns if the kmer_length is above 1.")
    elif explaining_params["kmer_length"] == 1:
        if config.get("sequence_column_name", None) is None and config.get("signal_column_name", None) is None:
            raise ValueError("There must be at least one sequence or score column if the kmer_length is 1.")

In [194]:
def create_heatmap(explaining_params):
    directory_name = f'{explaining_params["run_dir"]}'
    config = import_config(directory_name)
    model_explanation_out = f"{directory_name}/Results/Model_Explanation"
    
    # check if f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl' exists
    if not os.path.exists(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl'):
        raise ValueError(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl is prerequisite for the plotting.')
    
    full_shuffled_with_contribution_scores = pd.read_pickle(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl')
    
    subset_group_shuffled_with_contribution_scores = full_shuffled_with_contribution_scores[full_shuffled_with_contribution_scores[config["group_column_name"]] == explaining_params["group_to_explain"]]
    
    if not subset_group_shuffled_with_contribution_scores[config["group_column_name"]].isin([explaining_params["group_to_explain"]]).any():
        raise ValueError(f'{explaining_params["group_to_explain"]} is not a value in the {config["group_column_name"]} column.')
    
    sequences = subset_group_shuffled_with_contribution_scores[config["sequence_column_name"]].tolist()
    contribution_scores = subset_group_shuffled_with_contribution_scores[f'Contribution_Scores_for_Group_{explaining_params["group_to_explain"]}'].tolist()
    create_aggregate_heatmap_df_per_kmers(sequences, contribution_scores, explaining_params, model_explanation_out, config)

In [231]:
def plot_heatmap(explaining_params):
    directory_name = f'{explaining_params["run_dir"]}'
    config = import_config(directory_name)
    check_column_config(explaining_params, config)
    model_explanation_out = f"{directory_name}/Results/Model_Explanation"
    model_explanation_visualisation_out = create_model_explain_visualisation_dir(model_explanation_out)

    if not os.path.exists(f'{model_explanation_out}/heatmap_df.pkl'):
        raise ValueError(f'{model_explanation_out}/heatmap_df.pkl is prerequisite for the plotting.')
    
    heatmap_df = pd.read_pickle(f'{model_explanation_out}/heatmap_df.pkl')
    
    if explaining_params["plot_heatmap_and_clustermap"]:
        model_explanation_visualisation_heatmap_out = f"{model_explanation_visualisation_out}/Heatmap_and_Clustermap"
        os.makedirs(model_explanation_visualisation_heatmap_out, exist_ok=True)
        plot_a_heatmap_of_importance_scores(heatmap_df, model_explanation_visualisation_heatmap_out, explaining_params, config)
    if explaining_params["plot_sequences_clustered_by_bins"]:
        model_explanation_visualisation_heatmap_out = f"{model_explanation_visualisation_out}/Clustered_Heatmap"
        os.makedirs(model_explanation_visualisation_heatmap_out, exist_ok=True)
        plot_clustered_heatmap_of_importance_scores(heatmap_df, model_explanation_visualisation_heatmap_out, explaining_params, config)

In [112]:
def plot_individual_sequence_importance_scores(explaining_params):
    print("Plotting Individual Sequences...")
    directory_name = f'{explaining_params["run_dir"]}'
    config = import_config(directory_name)
    model_explanation_out = f"{directory_name}/Results/Model_Explanation"
    model_explanation_visualisation_out = create_model_explain_visualisation_dir(model_explanation_out)
    model_explanation_visualisation_individual_sequences_out = f"{model_explanation_visualisation_out}/Importance_Scores_Over_Sequences"
    os.makedirs(model_explanation_visualisation_individual_sequences_out, exist_ok=True)

    if not os.path.exists(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl'):
        raise ValueError(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl is prerequisite for the plotting.')
    
    full_shuffled_with_contribution_scores = pd.read_pickle(f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl')

    arrays_in_index_rows = full_shuffled_with_contribution_scores.loc[explaining_params["sequence_indexes"], f'Contribution_Scores_for_Group_{explaining_params["group_to_plot"]}'].tolist()

    for index, array_in_index_row in enumerate(arrays_in_index_rows):
        draw_importance_scores_values(array_in_index_row, index, model_explanation_visualisation_individual_sequences_out, explaining_params, config)

In [62]:
def run_modisco_lite(ohe_file, attr_file, explaining_params, output_path):
    
    print("Starting TF-MoDISco execution...")
    
    output_path_raw = f"{output_path}/Raw_Output"
    os.makedirs(output_path_raw, exist_ok=True)  
      
    # Construct the command
    output_file = f'{output_path_raw}/modisco_results_{explaining_params["group_to_extract"]}_{explaining_params["max_seqlets"]}_seqlets_{explaining_params["n_leiden"]}_leiden_{explaining_params["window"]}_window.h5'
    max_seqlets = str(explaining_params["max_seqlets"])
    n_leiden = str(explaining_params["n_leiden"])
    window = str(explaining_params["window"])

    if os.path.exists(output_file):
        return output_file
    
    command = [
        'modisco', 'motifs',
        '-s', ohe_file,
        '-a', attr_file,
        '-n', max_seqlets,
        '-l', n_leiden,
        '-w', window,
        '-o', output_file
    ]

    # Execute the command
    result = subprocess.run(command, capture_output=True, text=True)

    # Check for errors
    if result.returncode == 0:
        print("TF-MoDISco execution successful.")
    else:
        print(f"TF-MoDISco execution failed with error: {result.stderr}")
    return output_file
        
def visualise_modisco_lite(output_file, output_path, explaining_params):
    
    print("Starting TF-MoDISco visualisation...")
    
    output_path_visualised = f'{output_path}/Visualised_Output/{explaining_params["group_to_extract"]}_{explaining_params["max_seqlets"]}_seqlets_{explaining_params["n_leiden"]}_leiden_{explaining_params["window"]}_window'
    os.makedirs(output_path_visualised, exist_ok=True)  
    
    # Paths to input file and output directories
    input_file = output_file
    output_dir = output_path_visualised
    
    if os.path.exists(f'{output_dir}/motifs.html'):
        return 

    # Construct the command
    command = [
        'modisco', 'report',
        '-i', input_file,
        '-o', output_dir,
        '-s', output_dir
    ]

    # Execute the command
    result = subprocess.run(command, capture_output=True, text=True)

    # Check for errors
    if result.returncode == 0:
        print("TF-MoDISco report generation successful.")
    else:
        print(f"TF-MoDISco report generation failed with error: {result.stderr}")
        
    return 

In [249]:
def extract_motifs(explaining_params):
    print("Starting the modisco-lite motif extraction pipeline...")
    directory_name = f'{explaining_params["run_dir"]}'
    config = import_config(directory_name)
    if config.get("sequence_column_name", None) is None:
        raise ValueError("There must be a sequence column and present as the motifs will be extracted from the sequences.")
    model_explanation_out = f"{directory_name}/Results/Model_Explanation"
    model_explanation_visualisation_out = create_model_explain_visualisation_dir(model_explanation_out)
    model_explanation_visualisation_modisco_out = f"{model_explanation_visualisation_out}/TF_Modisco_Motif_Finder"
    os.makedirs(model_explanation_visualisation_modisco_out, exist_ok=True)
    
    modisco_input_path = f"{model_explanation_visualisation_modisco_out}/TF_Modisco_Input"
    os.makedirs(modisco_input_path, exist_ok=True)
    
    modisco_one_hot_encoded_input_path = f"{modisco_input_path}/onehot_data_for_modisco_{explaining_params['group_to_extract']}.npy"
    modisco_importance_scores_input_path = f"{modisco_input_path}/importance_scores_array_for_modisco_{explaining_params['group_to_extract']}.npy"
    
    original_file_shuffled_with_contribution_scores_path = f'{model_explanation_out}/original_file_shuffled_with_contribution_scores.pkl'

    if not os.path.exists(original_file_shuffled_with_contribution_scores_path):
        raise ValueError(f'{original_file_shuffled_with_contribution_scores_path} is prerequisite for the plotting.')
    
    full_shuffled_with_contribution_scores = pd.read_pickle(original_file_shuffled_with_contribution_scores_path)
    full_shuffled_with_contribution_scores = full_shuffled_with_contribution_scores[full_shuffled_with_contribution_scores[config["group_column_name"]] == explaining_params["group_to_extract"]]
    
    # Extract importance scores and find the maximum sequence length
    importance_scores = full_shuffled_with_contribution_scores[f'Contribution_Scores_for_Group_{explaining_params["group_to_extract"]}'].tolist()
    
    max_length = max(len(seq) for seq in importance_scores)
    
    importance_scores = pad_sequences(importance_scores, maxlen=max_length, dtype='float32', padding='post')
    importance_scores_array = np.array(importance_scores)
    
    # Truncate the importance scores to the first 4 columns as those are the seuqence columns
    importance_scores_array = importance_scores_array[:, :, :4]
    
    full_shuffled_with_contribution_scores['padded_onehot'] = full_shuffled_with_contribution_scores[config["sequence_column_name"]].apply(lambda x: one_hot_encode(x, max_length))
    onehot_data_array = np.stack(full_shuffled_with_contribution_scores['padded_onehot'].values)
    
    #importance_scores_array = importance_scores_array * onehot_data_array
    
    # Apply one-hot encoding and padding to the sequences
    if not os.path.exists(modisco_one_hot_encoded_input_path):
        print("Preparing the onehot data for modisco...")
        onehot_data_for_modisco = np.transpose(onehot_data_array, (0, 2, 1))
        np.save(modisco_one_hot_encoded_input_path, onehot_data_for_modisco)
        print(onehot_data_for_modisco.shape)
    
    if not os.path.exists(modisco_importance_scores_input_path):    
        print("Preparing the importance scores for modisco...")
        importance_scores_array_for_modisco = np.transpose(importance_scores_array, (0, 2, 1))
        np.save(modisco_importance_scores_input_path, importance_scores_array_for_modisco)
        print(importance_scores_array_for_modisco.shape)

    output_file = run_modisco_lite(modisco_one_hot_encoded_input_path, modisco_importance_scores_input_path, explaining_params, model_explanation_visualisation_modisco_out)
    visualise_modisco_lite(output_file, model_explanation_visualisation_modisco_out, explaining_params)

In [233]:
def add_predicitons(explaining_params):
    directory_name = f'{explaining_params["run_dir"]}'
    config = import_config(directory_name)

    column_indices, longest_sequence, categories_list = get_column_indices_max_length_and_categories(config)

    full_dataset = encode_from_csv(config['full_file'], config, column_indices, longest_sequence, categories_list).repeat()

    full_dataset_length = sum(1 for _ in open(config['full_file'])) - 1

    # Compute SHAP values for the desired number of batches
    num_batches_needed = (full_dataset_length // config['batch_size']) + 1

    if config['OPTIMISE']:
        model = load_model(f'{directory_name}/Results/Saved_Trained_Model_With_Best_Parameters/full_model.h5')
    else:
        model = load_model(f'{directory_name}/Results/Saved_Trained_Model/full_model.h5')

    # Predict using the model
    predictions = model.predict(full_dataset, steps=num_batches_needed)
    print("Predictions completed.")

    # Load original dataset to append predictions
    original_data = pd.read_csv(config['full_file'], sep=config['sep'])

    # Ensure `categories_list` is a flat list of strings
    categories_list = [cat.decode("utf-8") if isinstance(cat, bytes) else cat for cat in categories_list.numpy()]

    # Split predictions into separate columns based on `categories_list`
    for i, category in enumerate(categories_list):
        original_data[f'Prediction_{category}'] = predictions[:len(original_data), i]

    original_data.to_csv(f'{directory_name}/Results/Evaluated_Trained_Model/original_file_with_predictions.bed',
                         sep=config['sep'], index=False)


**-- CONFIG --**

In [238]:
config = {
    "TRAIN": True,
    "out_dir": "/ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models",
    "full_file": "/ceph/hpc/home/novljanj/data_storage/projects/semi_oops_project/Data/Naive_degs/Naive_transcripts/naive_degs_transcripts_with_fasta_good_collapsed_all_columns.bed",
    "sequence_column_name": "merged_sequence",
    "signal_column_name": ["clip", "SRSF3", "PABP1"],
    "group_column_name": "category",
    "sep": "\t",
    "test_and_validation_size": 0.15,
    "batch_size": 8,
    "num_blocks": 2,
    "dropout_prob": 0.1,"l2_lambda": 0.000,
    "reduce_by": 2,
    "Final_CNN_units": 100,
    "CNN_Units_Increase_By_Percent": 0.5,
    "Increase_Kernel_By": 1,
    "Increase_Dilation_By": 1,
    "LSTM_units": 50,
    "Dense_units": 50,
    "kernel_size": 5,
    "epochs": 200,
    "patience": 15,
    "learning_rate": 0.001,
    "normalization": None,
    "pooling_type": "max",
    
    "OPTIMISE": False,
    "n_trials": 100,
}

**-- TRAIN FROM SET CONFIG OR OPTIMISE THE CONFIG --**

In [239]:
def train_or_optimise(config):
    directory_name = prepare_the_necessary_directories_and_raw_files(config)
    training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape = prepare_the_data_for_training(config, directory_name)
    
    study = None
    if config['OPTIMISE']:
        study, best_params = optimise_with_optuna(config, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape)
        config.update(best_params)
        
    model, history = train_the_model(config, training_dataset, validation_dataset, training_steps, validation_steps, features_shape, labels_shape)
    evaluate_and_save_the_model(config, model, history, directory_name, validation_dataset, validation_steps, study)
    
if __name__ == "__main__":
    (config['TRAIN'] or config['OPTIMISE']) and train_or_optimise(config)


The longest sequence is 20771 nucleotides long.
Out dir: /ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models/Individual_Run_17052024_145717380


Mild Warning: The longest sequence is between 10,000 and 100,000 nucleotides long. This is not a problem but can cause extensive memory usage - check your available vram.


Estimated memory usage for a batch: 178 MB
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 20771, 6)]        0         
                                                                 
 conv1d_12 (Conv1D)          (None, 20771, 50)         1550      
                                                                 
 activation_12 (Activation)  (None, 20771, 50)         0         
                                                                 
 dropout_30 (Dropout)        (None, 20771, 50)         0         
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 10385, 50)        0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 10385, 100)        25100     
                

**-- CALCULATE THE IMPORTANCE SCORES FOR A GIVEN RUN AND/OR PLOT THEM--**

In [250]:
explaining_params = {

    "EXPLAIN": False, # Set to false if importance scores are already computed
    "run_dir": '/ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models/Individual_Run_17052024_145717380', # The output directory for the whole training run
    "num_background_samples": 200, # The number of samples to use for background in SHAP
    
    "ADD_PREDICTIONS": False, # Add the prediction for each sequence to the original file
    
    "PLOT_IMPORTANCE_SCORES_OVER_INDIVIDUAL_SEQUENCES": False, # Set to false if binned importance scores are already computed
    "group_to_plot": "OOPS & Semi-extractibilty DEGsn = 456", # The group to explain
    "sequence_indexes": [0], # Index of the sequence to plot from the original_file_shuffled, takes also lists of indexes
    
    "BIN_IMPORTANCE_SCORES": False, # Set to false if binned importance scores are already computed
    "group_to_explain": "OOPS & Semi-extractibilty DEGsn = 456", # The group to explain
    "kmer_length": 1, # The kmer to analyse 1=Nucleotide, 2=Dinucleotide, 3=Trinucleotide ...
    "num_bins": 10, # Make sure this is not much longer then the smallest sequences
    "plot_heatmap_and_clustermap": False, # Set to false if you don't want to plot the clustermap
    "plot_sequences_clustered_by_bins": False, # Set to false if you don't want to plot the sequences clustered based on their importance scores
    "type_of_reduction": "UMAP", # Choose from TSNE, UMAP, PCA, Isomap, MDS, Spectral Embedding, Factor Analysis, Dictionary Learning
    "number_of_clusters": 3, # Number of clusters formed after dim reduction
    
    "EXTRACT_MOTIFS_IN_IMPORTANCE_SCORES": True, # Set to false if binned importance scores are already computed
    "group_to_extract": "OOPS & Semi-extractibilty DEGsn = 456", # The group to explain
    "max_seqlets": 4000, # The maximum number of seqlets per metacluster.
    "n_leiden": 2, # The number of Leiden clusterings to perform with different random seeds.
    "window": 400, # The window surrounding the peak center that will be considered for motif discovery.
}

def explain_run(explaining_params):
    directory_name = f'{explaining_params["run_dir"]}'
    model_explanation_out = create_model_explain_out_dir(directory_name)
    config = import_config(directory_name)
    all_shap_values = compute_shaps(config, directory_name, explaining_params["num_background_samples"])
    add_shap_values_to_full_shuffled(all_shap_values, directory_name, config, model_explanation_out)
    
if __name__ == "__main__":
    explaining_params["ADD_PREDICTIONS"] and add_predicitons(explaining_params)
    explaining_params["EXPLAIN"] and explain_run(explaining_params)
    explaining_params["BIN_IMPORTANCE_SCORES"] and create_heatmap(explaining_params)
    (explaining_params["plot_heatmap_and_clustermap"] or  explaining_params["plot_sequences_clustered_by_bins"]) and plot_heatmap(explaining_params)
    explaining_params["PLOT_IMPORTANCE_SCORES_OVER_INDIVIDUAL_SEQUENCES"] and plot_individual_sequence_importance_scores(explaining_params)
    explaining_params["EXTRACT_MOTIFS_IN_IMPORTANCE_SCORES"] and extract_motifs(explaining_params)
    

Starting the modisco-lite motif extraction pipeline...
Starting TF-MoDISco execution...
TF-MoDISco execution failed with error: Traceback (most recent call last):
  File "/ceph/hpc/home/novljanj/.conda/envs/tf/bin/modisco", line 130, in <module>
    pos_patterns, neg_patterns = modiscolite.tfmodisco.TFMoDISco(
  File "/ceph/hpc/home/novljanj/.conda/envs/tf/lib/python3.9/site-packages/modiscolite/tfmodisco.py", line 339, in TFMoDISco
    neg_patterns = seqlets_to_patterns(seqlets=neg_seqlets,
  File "/ceph/hpc/home/novljanj/.conda/envs/tf/lib/python3.9/site-packages/modiscolite/tfmodisco.py", line 208, in seqlets_to_patterns
    cluster_indices = cluster.LeidenCluster(
  File "/ceph/hpc/home/novljanj/.conda/envs/tf/lib/python3.9/site-packages/modiscolite/cluster.py", line 22, in LeidenCluster
    partition = leidenalg.find_partition(
  File "/ceph/hpc/home/novljanj/.conda/envs/tf/lib/python3.9/site-packages/leidenalg/functions.py", line 81, in find_partition
    partition = partition_ty

In [247]:
# import a .npy file and check for NaN values
test = np.isnan(np.load('/ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models/Individual_Run_17052024_145717380/Results/Model_Explanation/Visualisations/TF_Modisco_Motif_Finder/TF_Modisco_Input/importance_scores_array_for_modisco_OOPS & Semi-extractibilty DEGsn = 456.npy')).any()
test
# import a .npy file and check for NaN values
test = np.isnan(np.load('/ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models/Individual_Run_17052024_145717380/Results/Model_Explanation/Visualisations/TF_Modisco_Motif_Finder/TF_Modisco_Input/onehot_data_for_modisco_OOPS & Semi-extractibilty DEGsn = 456.npy')).any()
test

False

In [248]:
test = np.load('/ceph/hpc/home/novljanj/data_storage/projects/ML-startup-guide/models/Individual_Run_17052024_145717380/Results/Model_Explanation/Visualisations/TF_Modisco_Motif_Finder/TF_Modisco_Input/onehot_data_for_modisco_OOPS & Semi-extractibilty DEGsn = 456.npy')
test

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.]]], dtype=float32)